<a href="https://colab.research.google.com/github/wgrh12/FluPredict/blob/master/uthAD_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## UTH Alzheimer's disease modeling of AD severity and separation from non-AD dementia using MethylSeq datasets in001, in027, in248 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
# Mount my Google Drive to allow access of files in it from colab
from google.colab import drive
drive.mount("/content/gdrive/",force_remount = True)

Mounted at /content/gdrive/


In [4]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data/UTH_AD"
! mkdir "/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD"

In [7]:
df_hyper = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/data/UTH_AD/2955_CpG_result_table_forADseverity.xls", sheet_name=0, convert_float=False)
df_hyper.shape

(2000, 137)

In [11]:
df_hyper_short = df_hyper.iloc[:,range(8,134,2)]
df_hyper_short.shape

(2000, 63)

In [13]:
df_hyper_short.head()

,70077027 meth_ratio,70077028 meth_ratio,70077054 meth_ratio,70077309 meth_ratio,70077012 meth_ratio,70077006 meth_ratio,70077078 meth_ratio,70077049 meth_ratio,70077010 meth_ratio,70077087 meth_ratio,70077031 meth_ratio,70077081 meth_ratio,70077016 meth_ratio,70077030 meth_ratio,70077009 meth_ratio,70077085 meth_ratio,70077042 meth_ratio,70077043 meth_ratio,70077039 meth_ratio,057 meth_ratio,029 meth_ratio,079 meth_ratio,050 meth_ratio,034 meth_ratio,95 meth_ratio,084 meth_ratio,74 meth_ratio,051 meth_ratio,053 meth_ratio,EPI-001 meth_ratio,EPI-008 meth_ratio,EPI-011 meth_ratio,EPI-012 meth_ratio,EPI-018 meth_ratio,EPI-020 meth_ratio,EPI-025 meth_ratio,EPI-027 meth_ratio,EPI-033 meth_ratio,EPI-047 meth_ratio,EPI-050 meth_ratio,EPI-051 meth_ratio,EPI-054 meth_ratio,EPI-063 meth_ratio,EPI-064 meth_ratio,EPI-080 meth_ratio,EPI-106 meth_ratio,EPI-108 meth_ratio,EPI-119 meth_ratio,EPI-135 meth_ratio,EPI-145 meth_ratio,EPI-002 meth_ratio,EPI-007 meth_ratio,EPI-009 meth_ratio,EPI-010 meth_ratio,EPI-013 meth_ratio,EPI-014 meth_ratio,EPI-023 meth_ratio,EPI-035 meth_ratio,EPI-048 meth_ratio,EPI-049 meth_ratio,EPI-052 meth_ratio,EPI-073 meth_ratio,EPI-077 meth_ratio
0,1.00,0.94,1.00,0.90,NaN,0.95,0.88,0.90,0.95,0.95,1.00,1.00,0.92,0.85,1.00,0.85,1.00,1.00,0.96,1.00,1.00,1.00,1.00,1.0,1.00,0.88,0.94,1.0,0.98,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.00,0.73,0.0,0.08,0.00,0.00,0.00,0.00,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.0,NaN,NaN,0.0,0.11
1,0.93,0.93,0.78,0.84,0.98,0.97,1.00,0.94,0.71,0.97,0.97,0.95,0.91,1.00,0.95,1.00,0.98,1.00,0.96,NaN,1.00,NaN,0.99,1.0,0.91,NaN,NaN,1.0,1.00,NaN,0.00,0.0,NaN,0.00,NaN,0.0,0.0,0.0,NaN,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,NaN,NaN,0.00,0.17,0.00,0.00,NaN,0.0,0.0,NaN,0.0,0.00
2,0.88,1.00,0.92,0.92,0.90,1.00,0.92,0.80,0.88,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.86,0.91,1.00,0.85,1.00,1.00,NaN,NaN,0.91,0.96,1.0,0.95,0.00,0.00,NaN,0.0,0.00,0.00,0.0,NaN,NaN,0.0,NaN,0.00,NaN,0.00,0.00,0.00,0.06,0.00,0.2,0.2,0.00,NaN,0.17,0.00,0.17,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.00
3,0.74,0.95,0.79,0.68,0.91,0.90,1.00,1.00,NaN,0.95,1.00,NaN,1.00,1.00,0.89,1.00,0.97,1.00,1.00,0.98,0.95,0.97,0.97,1.0,1.00,0.98,0.98,1.0,0.89,NaN,NaN,NaN,0.6,0.87,0.33,0.5,0.4,NaN,0.4,0.82,0.73,NaN,NaN,0.86,0.50,0.54,0.40,0.5,NaN,0.83,NaN,0.20,0.29,0.50,0.09,0.15,0.69,NaN,NaN,NaN,NaN,0.4,0.55
4,0.76,0.63,0.79,0.86,0.79,0.90,0.82,0.67,0.61,0.98,0.87,0.72,0.86,1.00,0.85,0.70,0.93,0.81,0.82,NaN,0.91,0.71,0.82,NaN,0.71,NaN,0.52,NaN,0.69,0.05,0.15,0.0,0.0,0.00,0.21,0.0,0.0,0.0,NaN,0.06,0.12,NaN,0.00,0.00,0.08,0.05,0.23,NaN,0.0,NaN,0.0,0.12,0.00,0.00,NaN,NaN,0.08,0.0,NaN,0.0,NaN,0.0,0.00


In [20]:
# Add row names
df_hyper_short['site'] = range(1,2001)
df_hyper_short.head()

,70077027 meth_ratio,70077028 meth_ratio,70077054 meth_ratio,70077309 meth_ratio,70077012 meth_ratio,70077006 meth_ratio,70077078 meth_ratio,70077049 meth_ratio,70077010 meth_ratio,70077087 meth_ratio,70077031 meth_ratio,70077081 meth_ratio,70077016 meth_ratio,70077030 meth_ratio,70077009 meth_ratio,70077085 meth_ratio,70077042 meth_ratio,70077043 meth_ratio,70077039 meth_ratio,057 meth_ratio,029 meth_ratio,079 meth_ratio,050 meth_ratio,034 meth_ratio,95 meth_ratio,084 meth_ratio,74 meth_ratio,051 meth_ratio,053 meth_ratio,EPI-001 meth_ratio,EPI-008 meth_ratio,EPI-011 meth_ratio,EPI-012 meth_ratio,EPI-018 meth_ratio,EPI-020 meth_ratio,EPI-025 meth_ratio,EPI-027 meth_ratio,EPI-033 meth_ratio,EPI-047 meth_ratio,EPI-050 meth_ratio,EPI-051 meth_ratio,EPI-054 meth_ratio,EPI-063 meth_ratio,EPI-064 meth_ratio,EPI-080 meth_ratio,EPI-106 meth_ratio,EPI-108 meth_ratio,EPI-119 meth_ratio,EPI-135 meth_ratio,EPI-145 meth_ratio,EPI-002 meth_ratio,EPI-007 meth_ratio,EPI-009 meth_ratio,EPI-010 meth_ratio,EPI-013 meth_ratio,EPI-014 meth_ratio,EPI-023 meth_ratio,EPI-035 meth_ratio,EPI-048 meth_ratio,EPI-049 meth_ratio,EPI-052 meth_ratio,EPI-073 meth_ratio,EPI-077 meth_ratio,site
0,1.00,0.94,1.00,0.90,NaN,0.95,0.88,0.90,0.95,0.95,1.00,1.00,0.92,0.85,1.00,0.85,1.00,1.00,0.96,1.00,1.00,1.00,1.00,1.0,1.00,0.88,0.94,1.0,0.98,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.00,0.73,0.0,0.08,0.00,0.00,0.00,0.00,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.0,NaN,NaN,0.0,0.11,1
1,0.93,0.93,0.78,0.84,0.98,0.97,1.00,0.94,0.71,0.97,0.97,0.95,0.91,1.00,0.95,1.00,0.98,1.00,0.96,NaN,1.00,NaN,0.99,1.0,0.91,NaN,NaN,1.0,1.00,NaN,0.00,0.0,NaN,0.00,NaN,0.0,0.0,0.0,NaN,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,NaN,NaN,0.00,0.17,0.00,0.00,NaN,0.0,0.0,NaN,0.0,0.00,2
2,0.88,1.00,0.92,0.92,0.90,1.00,0.92,0.80,0.88,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.86,0.91,1.00,0.85,1.00,1.00,NaN,NaN,0.91,0.96,1.0,0.95,0.00,0.00,NaN,0.0,0.00,0.00,0.0,NaN,NaN,0.0,NaN,0.00,NaN,0.00,0.00,0.00,0.06,0.00,0.2,0.2,0.00,NaN,0.17,0.00,0.17,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.00,3
3,0.74,0.95,0.79,0.68,0.91,0.90,1.00,1.00,NaN,0.95,1.00,NaN,1.00,1.00,0.89,1.00,0.97,1.00,1.00,0.98,0.95,0.97,0.97,1.0,1.00,0.98,0.98,1.0,0.89,NaN,NaN,NaN,0.6,0.87,0.33,0.5,0.4,NaN,0.4,0.82,0.73,NaN,NaN,0.86,0.50,0.54,0.40,0.5,NaN,0.83,NaN,0.20,0.29,0.50,0.09,0.15,0.69,NaN,NaN,NaN,NaN,0.4,0.55,4
4,0.76,0.63,0.79,0.86,0.79,0.90,0.82,0.67,0.61,0.98,0.87,0.72,0.86,1.00,0.85,0.70,0.93,0.81,0.82,NaN,0.91,0.71,0.82,NaN,0.71,NaN,0.52,NaN,0.69,0.05,0.15,0.0,0.0,0.00,0.21,0.0,0.0,0.0,NaN,0.06,0.12,NaN,0.00,0.00,0.08,0.05,0.23,NaN,0.0,NaN,0.0,0.12,0.00,0.00,NaN,NaN,0.08,0.0,NaN,0.0,NaN,0.0,0.00,5


In [22]:
# get df_hypo_short, and combine with df_hyper_short
df_hypo = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/data/UTH_AD/2955_CpG_result_table_forADseverity.xls", sheet_name=1, convert_float=False)
df_hypo_short = df_hypo.iloc[:,range(8,134,2)]
df_hypo_short['site'] = range(2001,4001)
df_2955 = df_hyper_short.append(df_hypo_short)
df_2955

,70077027 meth_ratio,70077028 meth_ratio,70077054 meth_ratio,70077309 meth_ratio,70077012 meth_ratio,70077006 meth_ratio,70077078 meth_ratio,70077049 meth_ratio,70077010 meth_ratio,70077087 meth_ratio,70077031 meth_ratio,70077081 meth_ratio,70077016 meth_ratio,70077030 meth_ratio,70077009 meth_ratio,70077085 meth_ratio,70077042 meth_ratio,70077043 meth_ratio,70077039 meth_ratio,057 meth_ratio,029 meth_ratio,079 meth_ratio,050 meth_ratio,034 meth_ratio,95 meth_ratio,084 meth_ratio,74 meth_ratio,051 meth_ratio,053 meth_ratio,EPI-001 meth_ratio,EPI-008 meth_ratio,EPI-011 meth_ratio,EPI-012 meth_ratio,EPI-018 meth_ratio,EPI-020 meth_ratio,EPI-025 meth_ratio,EPI-027 meth_ratio,EPI-033 meth_ratio,EPI-047 meth_ratio,EPI-050 meth_ratio,EPI-051 meth_ratio,EPI-054 meth_ratio,EPI-063 meth_ratio,EPI-064 meth_ratio,EPI-080 meth_ratio,EPI-106 meth_ratio,EPI-108 meth_ratio,EPI-119 meth_ratio,EPI-135 meth_ratio,EPI-145 meth_ratio,EPI-002 meth_ratio,EPI-007 meth_ratio,EPI-009 meth_ratio,EPI-010 meth_ratio,EPI-013 meth_ratio,EPI-014 meth_ratio,EPI-023 meth_ratio,EPI-035 meth_ratio,EPI-048 meth_ratio,EPI-049 meth_ratio,EPI-052 meth_ratio,EPI-073 meth_ratio,EPI-077 meth_ratio,site
0,1.00,0.94,1.00,0.90,NaN,0.95,0.88,0.90,0.95,0.95,1.00,1.00,0.92,0.85,1.00,0.85,1.00,1.00,0.96,1.00,1.00,1.00,1.00,1.00,1.00,0.88,0.94,1.00,0.98,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,0.00,0.00,0.73,0.00,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,NaN,0.00,0.11,1
1,0.93,0.93,0.78,0.84,0.98,0.97,1.00,0.94,0.71,0.97,0.97,0.95,0.91,1.00,0.95,1.00,0.98,1.00,0.96,NaN,1.00,NaN,0.99,1.00,0.91,NaN,NaN,1.00,1.00,NaN,0.00,0.00,NaN,0.00,NaN,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,0.00,0.17,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,2
2,0.88,1.00,0.92,0.92,0.90,1.00,0.92,0.80,0.88,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.86,0.91,1.00,0.85,1.00,1.00,NaN,NaN,0.91,0.96,1.00,0.95,0.00,0.00,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.00,NaN,0.00,NaN,0.00,0.00,0.00,0.06,0.00,0.20,0.20,0.00,NaN,0.17,0.00,0.17,NaN,NaN,NaN,0.00,0.00,NaN,0.00,NaN,0.00,3
3,0.74,0.95,0.79,0.68,0.91,0.90,1.00,1.00,NaN,0.95,1.00,NaN,1.00,1.00,0.89,1.00,0.97,1.00,1.00,0.98,0.95,0.97,0.97,1.00,1.00,0.98,0.98,1.00,0.89,NaN,NaN,NaN,0.60,0.87,0.33,0.50,0.40,NaN,0.40,0.82,0.73,NaN,NaN,0.86,0.50,0.54,0.40,0.50,NaN,0.83,NaN,0.20,0.29,0.50,0.09,0.15,0.69,NaN,NaN,NaN,NaN,0.40,0.55,4
4,0.76,0.63,0.79,0.86,0.79,0.90,0.82,0.67,0.61,0.98,0.87,0.72,0.86,1.00,0.85,0.70,0.93,0.81,0.82,NaN,0.91,0.71,0.82,NaN,0.71,NaN,0.52,NaN,0.69,0.05,0.15,0.00,0.00,0.00,0.21,0.00,0.00,0.00,NaN,0.06,0.12,NaN,0.00,0.00,0.08,0.05,0.23,NaN,0.00,NaN,0.00,0.12,0.00,0.00,NaN,NaN,0.08,0.00,NaN,0.00,NaN,0.00,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.12,0.12,0.57,0.00,0.44,0.20,0.22,0.08,0.17,0.43,0.22,0.00,0.12,0.25,0.07,0.40,0.10,0.22,0.25,NaN,0.00,0.00,NaN,NaN,NaN,0.33,NaN,NaN,0.00,0.27,0.00,0.13,0.31,0.46,0.18,0.07,NaN,0.10,0.00,0.00,0.00,NaN,0.00,0.11,0.50,0.14,0.00,0.05,0.78,0.00,0.16,0.35,0.23,0.22,0.38,0.17,0.28,NaN,0.25,0.20,0.67,0.35,0.45,3996
1996,0.75,NaN,NaN,0.38,0.50,0.90,0.78,NaN,0.50,0.25,0.57,NaN,0.50,0.67,0.86,NaN,NaN,NaN,0.71,NaN,0.62,0.43,0.88,0.60,0.62,0.67,0.40,0.50,0.72,0.78,0.23,0.61,0.57,0.50,0.63,0.67,0.54,0.75,0.69,0.82,0.29,0.90,0.80,0.47,0.71,0.79,0.61,0.80,0.64,0.88,0.77,0.76,0.70,0.67,0.60,0.50,0.92,NaN,0.88,NaN,1.00,0.83,0.75,3997
1997,0.33,0.50,0.39,0.71,0.54,0.49,0.80,0.53,0.60,0.35,0.69,0.50,0.53,0.27,0.44,0.38,0.46,0.41,0.46,1.00,0.44,0.60,0.62,0.03,0.70,0.75,0.42,0.00,0.54,0.38,0.50,0.32,0.50,0.35,0.43,0.38,0.20,0.40,0.42,0.64,0.27,0.24,0.29,0.31,0.23,0.20,0.24,0.33,0.53,0.55,0.52,0.90,0.75,0.50,0.83,0.35,0.26,0.69,0.64,0.63,0.50,0.32,0.61,3998
1998,0.50,0.30,0.25,0.14,0.11,0.00,0.33,0.00,0.14,0.30,0.22,0.36,0.50,0.00,0.0

In [23]:
df_2955.to_csv("/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD/AD_2955_ADvsNADD_top4000_sites.csv", index=False)

In [68]:
# Preprocessing data for modeling AD vs NADD, that is, non-AD dementia
X_nadd_raw = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/data/UTH_AD/AD_2955_ADvsNADD_top4000_sites_renamed_4ADseparation.csv", index_col=0)
y_nadd = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/data/UTH_AD/y_2955_AD_NADD_63samples.xlsx")
print(X_nadd_raw.shape)
y_nadd.shape  

(4000, 63)


(63, 1)

In [108]:
# Custom fill NAs with the mean of a particular group 
def fillNAs (datf):
  dfnew = datf
  splitnum = 50   # the sample split point of two groups
  # row by row
  for i in range(datf.shape[0]):
    mean_ad = datf.iloc[i,:splitnum].mean()
    #print(mean_ad)
    mean_nadd = datf.iloc[i,splitnum:].mean()
    # col by col
    for j in range(datf.shape[1]):
      check_NA = datf.iloc[i,:].isnull()
      if (check_NA[j]):
        if (j < splitnum):
          dfnew.iloc[i,j] = mean_ad
        else:
          dfnew.iloc[i,j] = mean_nadd 
  return dfnew


In [111]:
X_nadd = X_nadd_raw.dropna(thresh=60, inplace=False)
# Replace NAs with the mean of the row.
# X_nadd = X_nadd.T.fillna(X_nadd.mean(axis=1)).T , need understand it.
X_nadd_imputed = fillNAs( X_nadd )
X_nadd_imputed.to_csv("AD_2955_ADvsNADD_select708_sites_imputed.csv")
X_nadd = X_nadd_imputed.T
X_nadd.shape

(63, 708)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

In [113]:
X_nadd_train,X_nadd_test,y_nadd_train,y_nadd_test = train_test_split(X_nadd,y_nadd,stratify=y_nadd,random_state=42)
print(X_nadd_train.shape)
print(y_nadd_train.shape)
X_nadd_train.head()


(47, 708)
(47, 1)


site,6,44,46,57,59,75,120,144,147,197,205,243,255,287,307,310,321,331,335,342,343,344,345,355,365,366,368,390,392,395,396,404,428,432,438,440,445,447,457,460,...,3823,3826,3838,3846,3848,3854,3861,3862,3864,3865,3871,3878,3882,3890,3891,3892,3895,3899,3900,3907,3908,3912,3921,3928,3929,3936,3938,3942,3952,3956,3963,3973,3974,3975,3977,3978,3984,3990,3993,3998
in248_14,0.22,0.27,0.17,0.55,0.00,0.21,0.14,0.01,0.23,0.11,0.42,0.19,0.10,0.00,0.15,0.42,0.00,0.35,1.00,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.05,0.00,0.00,0.480000,0.00,0.24,...,0.85,0.80,1.00,1.00,0.36,0.57,0.80,0.83,1.00,0.30,0.78,0.86,1.00,0.40,0.35,0.90,0.60,0.20,0.60,0.46,0.67,1.0,0.30,0.11,0.19,0.59,0.50,0.88,0.00,0.21,0.42,0.00,0.75,0.60,0.33,0.14,0.98,0.89,0.50,0.35
in001_19,0.98,0.94,0.92,0.97,0.94,0.95,0.80,0.93,0.91,0.69,0.87,0.38,0.83,0.74,0.82,0.50,0.77,0.28,1.00,0.67,0.22,0.86,0.86,0.71,0.79,1.00,0.89,0.66,0.71,0.64,0.62,0.86,0.71,0.87,0.79,0.75,0.00,0.920000,0.58,0.80,...,0.57,0.27,0.80,0.83,0.40,0.11,0.69,0.67,0.64,0.19,0.70,0.80,0.82,0.12,0.33,0.71,0.20,0.25,0.43,0.54,0.50,1.0,0.12,0.57,0.29,0.31,0.80,0.33,0.20,0.40,0.56,0.00,0.50,0.71,0.08,0.29,0.71,0.92,0.25,0.41
in248_2,0.31,0.21,0.21,0.45,0.00,0.31,0.24,0.05,0.14,0.13,0.29,0.23,0.00,0.00,0.00,0.00,0.00,0.47,0.33,0.09,0.00,0.00,0.00,0.00,0.00,1.00,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.500000,0.00,0.30,...,0.86,0.36,0.79,0.86,0.25,0.48,1.00,0.87,0.86,0.39,1.00,0.86,0.89,0.14,0.67,0.79,0.28,0.14,0.43,0.49,0.44,1.0,0.43,0.40,0.28,0.74,0.67,0.14,0.38,0.35,0.75,0.21,1.00,0.73,0.22,0.55,1.00,0.67,0.40,0.52
in248_35,0.13,0.15,0.15,0.00,0.00,0.20,0.09,0.00,0.10,0.06,0.23,0.00,0.00,0.00,0.00,0.38,0.00,0.27,0.00,0.00,0.00,0.00,0.17,0.00,0.00,0.91,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.21,...,0.82,0.54,0.80,0.83,0.12,0.50,1.00,0.67,1.00,0.22,0.81,1.00,0.86,0.67,0.50,0.71,0.50,0.60,0.22,0.77,0.69,1.0,0.67,0.67,0.25,0.75,0.73,0.40,0.75,0.50,1.00,0.29,0.62,0.60,0.54,0.40,1.00,0.75,0.43,0.69
in001_14,0.90,0.96,0.87,0.97,0.92,1.00,0.78,0.95,0.81,0.59,0.78,0.82,0.75,0.50,0.50,0.86,0.67,0.18,1.00,0.82,0.86,0.56,0.60,0.64,0.58,1.00,0.62,0.50,0.59,0.57,0.65,0.44,0.80,0.75,0.88,0.62,0.62,0.562766,0.56,0.78,...,0.71,0.20,0.60,0.60,0.09,0.24,0.86,0.78,0.82,0.25,1.00,0.93,0.67,0.20,0.50,0.56,0.50,0.00,0.20,0.17,0.44,1.0,0.20,0.00,0.12,0.49,0.29,0.07,0.17,0.12,0.44,0.00,0.50,0.56,0.14,0.00,0.56,0.67,0.24,0.53


In [114]:
# Train logistic regression model.  Overfitted, want to select sites using LASSO   
params_grid = [{'C':[0.001,0.01,0.1,1,10,100]}]
grid_search = GridSearchCV(LogisticRegression(), params_grid, cv=5)
grid_search.fit(X_nadd_train,y_nadd_train)
print("Training R^2: {:.3f}".format(grid_search.score(X_nadd_train,y_nadd_train)))
print("Test R^2: {:.3f}".format(grid_search.score(X_nadd_test,y_nadd_test)))
print(grid_search.best_params_)
print(grid_search.best_score_)

Training R^2: 1.000
Test R^2: 1.000
{'C': 100}
1.0


In [131]:
# Feature selection using LASSO. Next, try to add age and gender as new features

y_nadd_train.loc[y_nadd_train['Status'] == "NADD", 'Status'] = 0
y_nadd_train.loc[y_nadd_train['Status'] == "AD", 'Status'] = 1
y_nadd_test.loc[y_nadd_test['Status'] == "NADD", 'Status'] = 0
y_nadd_test.loc[y_nadd_test['Status'] == "AD", 'Status'] = 1

params_grid = [{'alpha':[0.003,0.005,0.01,0.05,0.1,0.2]}]
grid_search = GridSearchCV(Lasso(), params_grid, cv=5)
grid_search.fit(X_nadd_train, y_nadd_train)
print("Training R^2: {:.3f}".format(grid_search.score(X_nadd_train,y_nadd_train)))
print("Test R^2: {:.3f}".format(grid_search.score(X_nadd_test,y_nadd_test)))
print(grid_search.best_params_)
print(grid_search.best_score_)

Training R^2: 0.987
Test R^2: 0.549
{'alpha': 0.003}
0.3599930383667228


In [132]:
! ls

AD_2955_ADvsNADD_select708_sites.csv	      gdrive
AD_2955_ADvsNADD_select708_sites_imputed.csv  sample_data


In [133]:
! cp AD* "/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD/"

In [135]:
# Build a lasso model using GridSearch results
lasso0003 = Lasso(alpha=0.003, random_state=42).fit(X_nadd_train, y_nadd_train)
print(lasso0003.score(X_nadd_train, y_nadd_train))
print(lasso0003.score(X_nadd_test, y_nadd_test))
print(np.sum(lasso0003.coef_ != 0))

0.9874265750059473
0.549108912263089
43


In [136]:
output = open("/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD/lasso0003_sites_.csv", "w")
for i in range(X_nadd_train.shape[1]):
  if lasso0003.coef_[i] != 0:
    print ("%s, %.5f" % (i, lasso0003.coef_[i]), file=output)
output.close()

In [139]:
output = open("/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD/lasso0003_sites_CpGs.csv", "w")
sites = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/output/UTH_AD/lasso0003_sites_.csv", header=None)
for i in range(sites.shape[0]):
  cpg = list(X_nadd_train.columns)[sites.iloc[i,0]]
  print ("%s" % cpg, file=output)
output.close()